# Okamoto-Tanaka Revisited protocol


In [1]:
import sys
print(sys.version)

3.5.3 (default, Sep 27 2018, 17:25:39) 
[GCC 6.3.0 20170516]


In [2]:
import random
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import math

KEY_SIZE = 512
H1_EXPONENT_SIZE = 256
SESSION_KEY_SIZE = 128
EPHEMERAL_EXPONENT_SIZE = 256
ID_SIZE = 64
ENDIAN = "little"


In [3]:
#dobór funkcji haschującej
def select_SHA_context(length_in_bits):
    if length_in_bits <= 224:
        return hashes.SHA224()
    elif length_in_bits <= 256:
        return hashes.SHA256()
    elif length_in_bits <= 384:
        return hashes.SHA384()
    elif length_in_bits <= 512:
        return hashes.SHA512()
    else:
        #TODO Exception
        print("Error, output to big for now")

#hf_KGC = select_SHA_context(H1_EXPONENT_SIZE)
#hf_USER = select_SHA_context(SESSION_KEY_SIZE)


In [4]:
def eea(_a,_b):
    u1 = 1
    v1 = 0
    u2 = 0
    v2 = 1
    if _a < 1 or _b < 1:
        return "Error"
    
    if _b>_a:
        a = _b
        b = _a
    else:
        a = _a
        b = _b
    r = _b
    while r!=0:
        g = r
        q = a//b
        
        r = a - (q*b)   
        u = u1 - (q*u2)
        v = v1 - (q*v2)
        u1 = u2
        v1 = v2
        u2 = u
        v2 = v
        
        a = b
        b = r
        
        #print(q,r,u,v)
    return (a, u1, v1)

#print(eea(77, 11))
#print(eea(62, 7))

## Generacja klucza

### Generacja klucza RSA dla KGC:

In [5]:
class KGC_key_public:
    def __init__(self, N, e, QRN_generator, h1sf, h2f, key_size):
        self.N = N
        self.e = e
        self.QRN_generator = QRN_generator
        self.h1sf = h1sf
        self.h2f = h2f
        self.key_size = key_size

class KGC_key:
    def __init__(self, h1sf, h2f,key_size = KEY_SIZE):
        
        rsa_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size = key_size,
            backend = default_backend()
        )
        self.key_size = key_size
        self.N = rsa_key.public_key().public_numbers().n
        self.e = rsa_key.public_key().public_numbers().e
        self.d = rsa_key.private_numbers().d
        p = rsa_key.private_numbers().p
        q = rsa_key.private_numbers().q
        self.Phi = (p-1)*(q-1)
        self.h1sf = h1sf # część funkcji hashującej H (wynik całej H musi być w QRN)
        self.h2f = h2f #druga funkcja hashująca jest w całości
    def set_QRN_generator(self, generator):
        self.QRN_generator = generator
    def public(self):
        #return (self.N,self.e,self.generator, self.h1sf, self.h2f)
        return KGC_key_public(self.N, self.e, self.QRN_generator, self.h1sf, self.h2f, self.key_size)
    # d,Phi - traktować jak prywatne

hash1_subfunction = select_SHA_context(H1_EXPONENT_SIZE)
hash2_function = select_SHA_context(SESSION_KEY_SIZE)
    
kgc01 = KGC_key(hash1_subfunction, hash2_function)
print('{:0128x}'.format(kgc01.N))
print('{:0128x}'.format(kgc01.e))
print('{:0128x}'.format(kgc01.d))
print(hex(kgc01.N))
        

dd040811d9274f919467d9db650c32602189de0633110f63ae3a09fa92e28ff7fa6a618571d4e851858ce923bc18b44c98abf85acab4e8c01876369660f9831f
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010001
197a747fb23be0e554696068f54e21a489593e5a129afdde2ce08df3efe8e455a80ac3fabb78f61d70f55d6c8e566cfe0471a63d55b699985041179c8852b9b1
0xdd040811d9274f919467d9db650c32602189de0633110f63ae3a09fa92e28ff7fa6a618571d4e851858ce923bc18b44c98abf85acab4e8c01876369660f9831f


In [6]:
def hashH1(material, kgc_public, endian = ENDIAN, id_size = ID_SIZE):
    #DEBUG_START
    print("hashH1: number to hash")
    print('{:08x}'.format(material))
    material = material.to_bytes(id_size//8, endian)
    #tp = [ord(material[i]) for i in range(material)]
    print("hashH1: bytes to hash:")
    print(material.hex())
    #print(tp)
    
    
    context = hashes.Hash(kgc_public.h1sf, backend=default_backend()) 
    context.update(material)
    hash_result = context.finalize()
    exponent = int.from_bytes(hash_result, byteorder = endian)
    print('Hash result:')
    print (hash_result.hex())
    print ('{:032x}'.format(exponent))
    #DEBUG_END
    hsh = pow(kgc_public.QRN_generator, exponent, kgc_public.N)
    return hsh


### Generacja kluczy uczestników mOT
#### Znalezienie generatora g grupy reszt kwadratowych mod N
Chcemy znaleźć generator podgrupy reszt kwadratowych w grupue Z_N. Oczekujemy, że grupa reszt będzie odpowiednio duża. Podgrupę będziemy oznaczać __QRN__ (analogicznie do oznaczenia w pracy Gannaro-Krawczyka-Rabina), generator będzie oznaczany __g__.
___
##### Rozkład Phi = d * (2^r) 
Najpierw wyznaczamy rozkład wartości funkcji Eulera dla _modułu klucza RSA_ __KGC__. Chcemy znaleźć takie __t__ oraz __r__, że __Phi = t * (2^r)__, przy czym __t__ jest nieparzyste.

In [7]:
#Rozłożenie N-1 = d* (2**r)
def mr(number):
    t = number
    r = 0
    while t%2 == 0:
    #while math.fmod(d,2) ==0:
        t//=2
        r+=1
    return (t,r)

(t,r) = mr(kgc01.Phi)
print(t,r)
print(t*(2**r))


723470874139470643460250174371654209360690340181140975029363522355482453201005158127379000080385927108174078296428911802912888741633256166412623435228681 4
11575533986231530295364002789946467349771045442898255600469816357687719251216082530038064001286174833730785252742862588846606219866132098662601974963658896


##### Wyznaczenie generatora g grupy QR_N
W celu wygenerowania akceptowalnego g losujemy pewnwe __alpha__ i sprawdzamy czy generuje podgrupę o satysfakcjonująco dużym rzędzie. Załóżmy tutaj, że zadowala nas jeżeli __alpha__ podniesione do potęgi t modulo N jest różne od 1.
W dalszej części podnosilibyśmy do kwadratu maksymalnie r razy. Jeżeli __alpha__ nie spełnia oczekiwań losujemy je ponownie i powtarzamy procedurę.  
_W przyszłości rozważyć szukanie większych podgrup (podnieść kilka razy do kwadratu i sprawdzić po ilu podniesieniach jest 1)._

In [8]:
#generator grupy Z_N: Znaleźć alpha że g = alpha**2 jest generatorem dużej podgrupy (reszt kwadratowych)
cont = True
while cont == True:
    alpha = random.randint(1,kgc01.N)
    #print (pow(alpha, Phi, N))
    beta = pow(alpha,t,kgc01.N)
    print (beta, alpha)
    if  beta != 1:
        cont = False
        #print ("alpha jest generatorem dużej podgrupy")
    
g = pow(alpha,2,kgc01.N)
kgc01.set_QRN_generator(g)
print (pow(kgc01.QRN_generator, kgc01.Phi, kgc01.N))

4654023467529818705425778885694696827712508556039352252316103100611824056096275656912923185600269597557176280338552241865162662055787565298210452238139541 2908650002741944830717752395733969395339256606085773748289961042778675725796306303575263914620097812286037858245141253557083071685109577005852557572774725
1


#### Klasa użytkowników

In [9]:
#Użytkownik
class User:
    def __init__(self,name,user_id):
        self.name = name
        self.id = user_id #user_id.to_bytes(ID_SIZE//8, byteorder=ENDIAN)
        #self.idn = int.from_bytes(self.id, byteorder=ENDIAN)
        
    def set_private_key(self,key, endian = ENDIAN):
        self.private_key = key
        #self.private_key_n = int.from_bytes(key, endian)
    def generate_ephemeral_exponent(self, ephemeral_exponent_size = EPHEMERAL_EXPONENT_SIZE):
        self.ephemeral_exponent = random.randint(2**(ephemeral_exponent_size//4),2**(ephemeral_exponent_size))
    def calculate_shared_secred(self, msg, id_corr, kgc_public, endian = ENDIAN):
        #print("id_corr", id_corr)
        #hsh = hashH1(corr_idn.to_bytes(kgc_public.key_size//8, endian), kgc_public)
        hsh = hashH1(id_corr, kgc_public)
        rev_hsh = eea(kgc_public.N, hsh)[2]
        
        print("with reversing hsh:", hsh*rev_hsh%kgc_public.N)
        
        
        
        #msgn = int.from_bytes(msg,endian)
        #msgn_e = pow(msgn, kgc_public.e, kgc_public.N)
        msgn_e = pow(msg, kgc_public.e, kgc_public.N)
        
        t = (msgn_e * rev_hsh)%kgc_public.N
        K = pow(t, 2*self.ephemeral_exponent, kgc_public.N)
        
        return K
        
        
    
user_A = User("A",random.randint(1,2**ID_SIZE))
user_B = User("B",random.randint(1,2**ID_SIZE))

#### Generacja kluczy prywatnych uczestników mOT.

In [10]:
# funkcja hashująca do grupy reszt kwadratowych
def generate_user_ltk(kgc, user_id, endian = ENDIAN):
    #context = hashes.Hash(kgc.h1sf, backend=default_backend()) 
    #context.update(material)
    #exponent = int.from_bytes(context.finalize(), byteorder = endian)
    #sk = pow(kgc.QRN_generator, exponent, kgc01.N)
    hsh = hashH1(user_id, kgc.public(), endian)
    return pow(hsh, kgc.d, kgc.N)

user_A.set_private_key(generate_user_ltk(kgc01, user_A.id))
user_B.set_private_key(generate_user_ltk(kgc01, user_B.id))
print((user_A.private_key))
print('{:08x}'.format(user_A.private_key))
    

    

hashH1: number to hash
a2a5e0c9e7c6f76c
hashH1: bytes to hash:
6cf7c6e7c9e0a5a2
Hash result:
f5eb6f1ce584d2ba25349f301d9916cb6085cfc060635cdb5fd2497f8eace13d
3de1ac8e7f49d25fdb5c6360c0cf8560cb16991d309f3425bad284e51c6febf5
hashH1: number to hash
7d42c642e8c3935c
hashH1: bytes to hash:
5c93c3e842c6427d
Hash result:
5e24ebf615adeb9aa9df6bd0dcd7a40b4acc78283c1cae84a1e695ca946c5aed
ed5a6c94ca95e6a184ae1c3c2878cc4a0ba4d7dcd06bdfa99aebad15f6eb245e
6524318363606528881225417242754971872947111333962656479778917185867290316018517401697143894332574316671000523868842509002958412700085447106323373020957040
7c9234308a01b505a17c8b221da4bccb4212bc9971bde25315a34db21a003aec63c1897b9b018bdfcf84e7582b56da65924bd8ca6a042c359e32eb38aef84970


##### Weryfikacja klucza 

In [11]:
usk = user_A.private_key
usk_pow_e = pow(usk,kgc01.e,kgc01.N)
#usk_pow_e = usk
hash_context = hashes.Hash(select_SHA_context(H1_EXPONENT_SIZE), backend=default_backend()) 
hash_context.update(user_A.id.to_bytes(ID_SIZE, byteorder = ENDIAN))
exponent = int.from_bytes(hash_context.finalize(), byteorder = ENDIAN)
sk_pre_d = pow(kgc01.QRN_generator, exponent, kgc01.public().N)
print ( sk_pre_d == usk_pow_e)
if(sk_pre_d != usk_pow_e):
    print (usk_pow_e,"\n", sk_pre_d )
    




False
3328599544704344042077349624901703801253656395840193612775323583568869096361669166730935519816367967405737938987953000251801969519175515841346898136952347 
 2553810652737459162170475961518562409113370500155778590955941606634383161482860720165206978046751718943431296590506762088273496300254993678322854132021138


## Przebieg sesji

### Generacja wiadomości


In [12]:

def generate_negotiation_message(user, kgc_public, endian = ENDIAN):
    g_p = pow(kgc_public.QRN_generator, user.ephemeral_exponent, kgc_public.N)
    return ((g_p*user.private_key)%kgc_public.N)
    

user_A.generate_ephemeral_exponent()
user_B.generate_ephemeral_exponent()

msg_A = generate_negotiation_message(user_A,kgc01.public())
msg_B = generate_negotiation_message(user_B,kgc01.public())

#print(msg_A)

    

#idA = bytes(user_A.id.to_bytes(ID_SIZE, byteorder="little"))
#idA = bytes(user_A.id.to_bytes(ID_SIZE, byteorder="big"))
print(user_A.id)



#print(idA)
#print(hex(user_A.id))

11720020763083208556


In [13]:
k_A = user_A.calculate_shared_secred(msg_B, user_B.id, kgc01.public())
k_B = user_B.calculate_shared_secred(msg_A, user_A.id, kgc01.public())

k = pow(kgc01.QRN_generator,2*user_A.ephemeral_exponent*user_B.ephemeral_exponent*kgc01.e, kgc01.N)
#print (kgc01.QRN_generator)
print("k=\t", k)
print("k_A=\t", k_A)
print("k_B=\t", k_B)

k1a = pow(msg_B, kgc01.e, kgc01.N)
k2a = eea(kgc01.N,hashH1(user_B.id, kgc01.public()))[2]
k3a = k1a*k2a%kgc01.N
k4a = pow(k3a, 2*user_A.ephemeral_exponent, kgc01.N)
print("k1a =\t", k1a)
print("k2a =\t", k2a)
print("k3a =\t", k3a)
print("k4a =\t", k4a)



hashH1: number to hash
7d42c642e8c3935c
hashH1: bytes to hash:
5c93c3e842c6427d
Hash result:
5e24ebf615adeb9aa9df6bd0dcd7a40b4acc78283c1cae84a1e695ca946c5aed
ed5a6c94ca95e6a184ae1c3c2878cc4a0ba4d7dcd06bdfa99aebad15f6eb245e
with reversing hsh: 1
hashH1: number to hash
a2a5e0c9e7c6f76c
hashH1: bytes to hash:
6cf7c6e7c9e0a5a2
Hash result:
f5eb6f1ce584d2ba25349f301d9916cb6085cfc060635cdb5fd2497f8eace13d
3de1ac8e7f49d25fdb5c6360c0cf8560cb16991d309f3425bad284e51c6febf5
with reversing hsh: 1
k=	 7446352313258992064937674073790260779821089131848212310198940688890441025303766442723191472829619181730613337336767819615569096221881195751931240919405703
k_A=	 7446352313258992064937674073790260779821089131848212310198940688890441025303766442723191472829619181730613337336767819615569096221881195751931240919405703
k_B=	 7446352313258992064937674073790260779821089131848212310198940688890441025303766442723191472829619181730613337336767819615569096221881195751931240919405703
hashH1: number to hash
7d42c6